# LangChaing version: Questions from YT transcript video

In [1]:
import os
import openai
import sys
sys.path.append('../..')
from openai import AzureOpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file


AZURE_OPENAI_ENDPOINT_GPT = os.getenv('AZURE_OPENAI_ENDPOINT_GPT')
AZURE_OPENAI_ENDPOINT_EMBEDDING = os.getenv('AZURE_OPENAI_ENDPOINT_EMBEDDING')
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION')
OPENAI_MODEL_NAME = os.getenv('OPENAI_MODEL_NAME')
OPENAI_GPT4_TURBO_ENGINE = os.getenv('OPENAI_GPT4_TURBO_ENGINE')

### 1)Document loading

In [2]:
from langchain_community.document_loaders import TextLoader
text = 'Ramilla-Mauritania-1.txt'
loader = TextLoader(text)
pages = loader.load()
x = pages[0].page_content

### 2)Creating the model

In [3]:
from langchain_openai import AzureChatOpenAI

In [5]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
llm_model = 'gpt-4o'
chat = AzureChatOpenAI(temperature=0.0, azure_deployment=llm_model, api_version='2024-02-15-preview',azure_endpoint = AZURE_OPENAI_ENDPOINT_GPT)


Creating a template for the prompt, to repeatly use it

In [12]:
template_string = """Extrae los datos mas interesantes \
del texto delimitado por triple comas \
, el texto trata sobre el script de un video de YouTube llamado {youtube_video_name}\
el output debe ser un diccionario key valor \
text: ```{text}```
"""

we need to create an instance of the class ChatPromptTemplate

In [13]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [14]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['text', 'youtube_video_name'], input_types={}, partial_variables={}, template='Extrae los datos mas interesantes del texto delimitado por triple comas , el texto trata sobre el script de un video de YouTube llamado {youtube_video_name}el output debe ser un diccionario key valor text: ```{text}```\n')

In [15]:
prompt_template.messages[0].prompt.input_variables

['text', 'youtube_video_name']

Now we fille each variable in the template (text and youtube_video_name)

In [16]:
# Style, which we want our sentence to be transformed
youtube_transcript_text = x

#Text, text we wanna transform to a specific style..
youtube_vid_name = """Ramilla de Aventuras en maurationa\
"""

In [18]:
# The custom message will generate the prompt and send it to a LLM model
# to get a respond

customer_messages = prompt_template.format_messages(
                    text=youtube_transcript_text,
                    youtube_video_name=youtube_vid_name)

#print(customer_messages)

Now, let´s call the LLM for the execution of the prompt template

In [19]:
# Call the LLM to translate to the style of the customer message
# We send the promt created, through 
customer_response = chat(customer_messages)

/tmp/ipykernel_1587/4274786145.py:3: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  customer_response = chat(customer_messages)


In [29]:
#print(customer_response.content)

Now let´s focus on Parsers, Parser is a class in LangChan that allows to specify how the output shuld be.

In [25]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [27]:
# We define the schema we want for the outputs, then we introduce them into a list. 

video_name_schema = ResponseSchema(name="video_name",
                             description="Cual es el nombre del video\
                                        de donde se esta extrayendo la información\
                                            en el caso de no estar presente la respuesta debe ser\
                                                'desconocido'")
video_interesting_facts_schema = ResponseSchema(name="interesting_facts",
                                      description="Debe ser una lista con todos los datos interesantes\
                                        que se puedan extraer del script del video.")
country_schema = ResponseSchema(name="pais",
                                    description="Cual es el nombre del pais que se visita en el video\
                                            en el caso de no estar presente la respuesta debe ser\
                                                'desconocido")

response_schemas = [video_name_schema, 
                    video_interesting_facts_schema,
                    country_schema]

In [28]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"video_name": string  // Cual es el nombre del video                                        de donde se esta extrayendo la información                                            en el caso de no estar presente la respuesta debe ser                                                'desconocido'
	"interesting_facts": string  // Debe ser una lista con todos los datos interesantes                                        que se puedan extraer del script del video.
	"pais": string  // Cual es el nombre del pais que se visita en el video                                            en el caso de no estar presente la respuesta debe ser                                                'desconocido
}
```


In [35]:
template_string_2 = """Extrae los datos mas interesantes \
del texto delimitado por triple comas \
, el texto trata sobre el script de un video de YouTube llamado {youtube_video_name}\
el output debe ser un diccionario key valor \
text: ```{text}```

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=template_string_2)

messages = prompt.format_messages(text=x, 
                                youtube_video_name ='Ramilla de AVenturas mauritania',
                                format_instructions=format_instructions)

#We send the prompt to the LLM
response = chat(messages)

In [36]:
print(response.content)

```json
{
	"video_name": "Ramilla de AVenturas mauritania",
	"interesting_facts": [
		"La capital de Mauritania es Nouakchott.",
		"El tren del desierto, conocido como el tren del Sáhara, es una atracción importante.",
		"Mauritania es uno de los países menos visitados de África y del mundo.",
		"Los taxis en Nouakchott funcionan de manera informal, preguntando a los conductores si quieren llevarte.",
		"No hay rascacielos en Nouakchott.",
		"El mercado central de Nouakchott tiene mucha arena, similar a una playa.",
		"Muchos vendedores ambulantes son senegaleses y malienses.",
		"Los coches más comunes en Nouakchott son Toyota, aunque hay algunos Mercedes.",
		"La empresa minera que opera el tren del desierto tiene su sede en Nouakchott.",
		"El país es mayormente desierto, lo que dificulta la vida fuera de las ciudades.",
		"Nouakchott fue diseñada para 8,000 personas, pero la población creció debido a sequías en los años 70 y 80.",
		"El transporte público en Mauritania es caótico, 

In [37]:
# We parse the output into a dict 

output_dict = output_parser.parse(response.content)

In [38]:
output_dict

{'video_name': 'Ramilla de AVenturas mauritania',
 'interesting_facts': ['La capital de Mauritania es Nouakchott.',
  'El tren del desierto, conocido como el tren del Sáhara, es una atracción importante.',
  'Mauritania es uno de los países menos visitados de África y del mundo.',
  'Los taxis en Nouakchott funcionan de manera informal, preguntando a los conductores si quieren llevarte.',
  'No hay rascacielos en Nouakchott.',
  'El mercado central de Nouakchott tiene mucha arena, similar a una playa.',
  'Muchos vendedores ambulantes son senegaleses y malienses.',
  'Los coches más comunes en Nouakchott son Toyota, aunque hay algunos Mercedes.',
  'La empresa minera que opera el tren del desierto tiene su sede en Nouakchott.',
  'El país es mayormente desierto, lo que dificulta la vida fuera de las ciudades.',
  'Nouakchott fue diseñada para 8,000 personas, pero la población creció debido a sequías en los años 70 y 80.',
  'El transporte público en Mauritania es caótico, con autobuses

In [20]:
#! pip install python-docx

Below is the code for saving each value extracted from the YT transcrip.

In [41]:
from docx import Document

# Create a new Document
doc = Document()

# Add a title
doc.add_heading('Ramilla de Aventuras Mauritania_LangChain_Version', level=1)

# Add a paragraph
doc.add_paragraph('Resumen del video.')

for i in list(output_dict.keys()):
# Add a bullet point list
    doc.add_paragraph(output_dict[i], style='ListBullet')


# Save the document
doc.save('Ramilla_mauritania_v2.docx')

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/docx/styles/styles.py:130: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)
